In [1]:
import os
creadentials_path = "C:/Users/poornas/Downloads/private_key.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = creadentials_path

In [2]:
from google.cloud import bigquery

In [3]:
client = bigquery.Client()
project = client.project
dataset_ref = bigquery.DatasetReference(project, 'adventus')
table_id = "adventus-338217.adventus.customers"

In [4]:
# this code create the table manually
schema = [
    bigquery.SchemaField("_id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("index", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("guid", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("isActive", "BOOL", mode="NULLABLE"),
    bigquery.SchemaField("age", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("eyeColor", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("name", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("gender", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("company", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("email", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("phone", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("address", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("about", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("registered", "TIMESTAMP", mode="NULLABLE"),
    bigquery.SchemaField("latitude", "FLOAT64", mode="NULLABLE"),
    bigquery.SchemaField("longitude", "FLOAT64", mode="NULLABLE"),
    bigquery.SchemaField("tags", "STRING", mode="REPEATED"),
    bigquery.SchemaField(
        "friends",
        "RECORD",
        mode="REPEATED",
        fields=[
            bigquery.SchemaField("id", "INTEGER", mode="NULLABLE"),
            bigquery.SchemaField("name", "STRING", mode="NULLABLE"),
        ],
    ),
    bigquery.SchemaField("greeting", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("favoriteFruit", "STRING", mode="NULLABLE"),
]
table_ref = dataset_ref.table("customers")
table = bigquery.Table(table_ref, schema=schema)
table = client.create_table(table)  # API request

print("Created table {}".format(table.full_table_id))

Created table adventus-338217:adventus.customers


In [5]:
#Adding customers_1.json data into previously created table
job_config = bigquery.LoadJobConfig(
    schema=schema,
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
)
uri = "https://storage.cloud.google.com/bucket_advantus/customers_1.json"

load_job = client.load_table_from_uri(
    uri,
    table_id,
    location="US",  
    job_config=job_config,
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(table_id)
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 4 rows.


In [6]:
#adding customers_2.json data into the table with schema changes
job_config = bigquery.LoadJobConfig(
    autodetect=True, source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
)
job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND
job_config.schema_update_options = [
    bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION
]
uri = "https://storage.cloud.google.com/bucket_advantus/customers_2.json"
load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.
load_job.result()  # Waits for the job to complete.
destination_table = client.get_table(table_id)
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 8 rows.


In [9]:
""" 
Streaming insert the customers_3.json file....
Had to do two changes. 
 1) Set ignore_unknown_values to Ture. So, this will ignore new columns which available in steaming dataset. 
 2) The column named “Balance” automatically converted to float when loading the second file. But before loading the 3rd file I had to remove the "$" sign and the comma manually from that column. 

 """ 

rows_to_insert = [
    {"_id":"61b0d935fbdfc2400db7ca6d","index":0,"guid":"3d42d8df-128f-431e-81c9-02e521162af1","isActive":"true","balance":"2132.94","picture":"http://placehold.it/32x32","age":38,"eyeColor":"blue","name":"Adrienne Anderson","gender":"female","company":"JIMBIES","email":"adrienneanderson@jimbies.com","phone":"+1 (996) 568-2799","address":"878 Chester Avenue, Vicksburg, Puerto Rico, 9939","about":"Dolore quis id enim qui eiusmod. Laborum nostrud enim do elit sunt enim. Aute labore enim duis tempor cillum nisi officia voluptate. Esse non qui consequat in laboris culpa cillum eu deserunt anim. Lorem nulla magna cillum reprehenderit sunt esse irure nisi.\r\n","registered":"2016-10-21T07:00:38 -01:00","latitude":5.417943,"longitude":-18.414087,"tags":["est","aliquip","ea","commodo","dolore","duis","nulla"],"friends":[{"id":0,"name":"Bass Kirkland"},{"id":1,"name":"Hoffman Zimmerman"},{"id":2,"name":"Rosario Caldwell"}],"foes":[{"id":0,"name":"Kerr Howard"},{"id":1,"name":"Robles Albert"},{"id":2,"name":"Georgia Hurst"}],"greeting":"Hello, Adrienne Anderson! You have 6 unread messages.","favoriteFruit":"banana"},
{"_id":"61b0d93562d87ce4371a0f33","index":1,"guid":"c7c2463c-6984-488f-8e2c-8a9ae6d430b4","isActive":"false","balance":"2509.43","picture":"http://placehold.it/32x32","age":21,"eyeColor":"green","name":"Sampson Johnston","gender":"male","company":"ASIMILINE","email":"sampsonjohnston@asimiline.com","phone":"+1 (829) 564-2120","address":"322 Dorchester Road, Dixie, Hawaii, 1738","about":"Lorem exercitation eiusmod ullamco aliquip enim ex excepteur adipisicing. Consectetur dolore esse sunt ea velit dolor labore labore laborum velit. Ipsum occaecat ad dolore fugiat commodo amet consectetur culpa. Nostrud sunt officia aute labore commodo irure duis. Eu adipisicing est fugiat fugiat excepteur labore exercitation duis. Esse ad minim nulla veniam veniam dolore sunt laborum quis nulla magna cupidatat cillum in. Officia nulla ipsum eu adipisicing et proident ullamco labore amet nulla consectetur eiusmod pariatur.\r\n","registered":"2017-11-03T07:12:14 -00:00","latitude":51.988023,"longitude":-14.471322,"tags":["mollit","in","cupidatat","occaecat","qui","in","labore"],"friends":[{"id":0,"name":"Pitts Wise"},{"id":1,"name":"French Hurley"},{"id":2,"name":"Rodriquez Chen"}],"foes":[{"id":0,"name":"Amanda Franco"},{"id":1,"name":"Henrietta Estes"},{"id":2,"name":"Hurst Hester"}],"greeting":"Hello, Sampson Johnston! You have 4 unread messages.","favoriteFruit":"strawberry"},
{"_id":"61b0d935314e426e37546087","index":2,"guid":"c47cfc9c-98e7-43f9-9c64-9af97eec6bd7","isActive":"true","balance":"1673.53","picture":"http://placehold.it/32x32","age":36,"eyeColor":"blue","name":"Grace Trujillo","gender":"female","company":"HYDROCOM","email":"gracetrujillo@hydrocom.com","phone":"+1 (978) 415-3940","address":"338 Anthony Street, Garfield, Pennsylvania, 3456","about":"Amet consectetur commodo qui occaecat amet aliquip eiusmod aliqua ullamco labore ad. Ut et veniam duis id. Ipsum anim mollit laboris dolore adipisicing enim non esse culpa commodo. Ipsum qui enim esse ex labore culpa id incididunt duis do sint magna pariatur. Nostrud elit aute tempor esse ut anim Lorem ex occaecat exercitation dolor. Ipsum quis velit sint consequat anim sit non Lorem enim ex velit sit incididunt.\r\n","registered":"2019-08-13T01:42:34 -01:00","latitude":16.870529,"longitude":138.096246,"tags":["ullamco","anim","officia","cillum","sint","quis","reprehenderit"],"friends":[{"id":0,"name":"Nielsen Curtis"},{"id":1,"name":"Therese Chang"},{"id":2,"name":"Bernadine Callahan"}],"foes":[{"id":0,"name":"Battle Camacho"},{"id":1,"name":"Moreno Morton"},{"id":2,"name":"Morrow Hyde"}],"greeting":"Hello, Grace Trujillo! You have 6 unread messages.","favoriteFruit":"strawberry"},
{"_id":"61b0d9353bbf11740c5cfa05","index":3,"guid":"850bb4ee-cd99-49b0-9e0e-af5cff5127c8","isActive":"true","balance":"1650.41","picture":"http://placehold.it/32x32","age":26,"eyeColor":"green","name":"Shelly Little","gender":"female","company":"EVEREST","email":"shellylittle@everest.com","phone":"+1 (871) 515-2194","address":"886 Moultrie Street, Dowling, Vermont, 6379","about":"Sint eiusmod sint incididunt eu aute exercitation. Ipsum officia aliquip irure et veniam commodo reprehenderit laboris eiusmod voluptate cillum. Ipsum reprehenderit eu minim proident nostrud occaecat in sit nulla deserunt ipsum quis aute laborum. Minim esse incididunt aliquip aliquip irure ea magna anim duis occaecat. Ad excepteur laboris quis id commodo et velit excepteur sit velit nostrud tempor aute.\r\n","registered":"2014-11-15T04:39:11 -00:00","latitude":-45.157174,"longitude":-106.304008,"tags":["sunt","ipsum","mollit","exercitation","minim","occaecat","nostrud"],"friends":[{"id":0,"name":"Jeannette Frederick"},{"id":1,"name":"Hollie Lamb"},{"id":2,"name":"Kirkland Olsen"}],"foes":[{"id":0,"name":"Garza Long"},{"id":1,"name":"Tamera Logan"},{"id":2,"name":"Kelli Fulton"}],"greeting":"Hello, Shelly Little! You have 5 unread messages.","favoriteFruit":"strawberry"},
]

errors = client.insert_rows_json(table_id, rows_to_insert, ignore_unknown_values=True)  # Make an API request.
if errors == []:
    print("New rows have been added.")
else:
    print("Encountered errors while inserting rows: {}".format(errors))

New rows have been added.
